In [ ]:
import os, re, csv
import pandas as pd
import numpy as np
import time

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction import text


In [ ]:
# Created in MIMIC_004_keyword_prevalence-per_patient.ipynb
#all_patients0 = pd.read_csv('data/keyword_prevalence_per_patient_nodummies_10.12.csv')
all_patients0 = pd.read_csv('data/keyword_prevalence_per_patient_10.12.csv',
                           low_memory = False)

In [ ]:
print(all_patients0.columns.tolist())

In [ ]:
print(all_patients0.shape)
all_patients=all_patients0[all_patients0.CATEGORY!='Discharge summary']
print(all_patients.shape)
# 7,667 patients as expected.
print("Unique patients:", all_patients.SUBJECT_ID.nunique())

In [ ]:
all_patients['LENGTH_OF_STAY']=pd.to_timedelta(all_patients['LENGTH_OF_STAY'])
all_patients['day_of_note_from_admit']=pd.to_datetime(all_patients['CHARTTIME']).dt.date-pd.to_datetime(all_patients['ADMITTIME']).dt.date
all_patients['day_of_note_from_admit']=all_patients['day_of_note_from_admit'].fillna(all_patients['LENGTH_OF_STAY'])

In [ ]:
#all_patients[['ADMITTIME', 'DISCHTIME', 'CHARTTIME', 'CATEGORY', 'day_of_note_from_admit']]

In [ ]:
#Check number of positive keywords is groupd by patient and day of note from admission
all_patients.groupby(['SUBJECT_ID', 'HADM_ID', 'day_of_note_from_admit'])['NUM_POS_KEYWORDS'].sum()

In [ ]:
#Load pre-determined keywords
all_keywords=[]
neg_keywords=pd.read_csv('data-raw/negative_keywords.csv').iloc[::,0]
pos_keywords=pd.read_csv('data-raw/positive_keywords.csv').iloc[::,0]

all_keywords.append(neg_keywords.apply(lambda x: x.strip()).tolist())
all_keywords.append(pos_keywords.apply(lambda x: x.strip()).tolist())

# Convert to a dictionary temporarily to deduplicate keyword list ("concerning" is duplicated)
keywords= list(dict.fromkeys([i.lower() for i in all_keywords for i in i]))
negative_keywords=[i.strip().lower() for i in neg_keywords]
positive_keywords=[i.strip().lower() for i in pos_keywords]

In [ ]:
print("Total keywords:", len(keywords))
print(", ".join(keywords))

In [ ]:
%%time
# Takes 2 mins

#finding Tfidf score per keyword 
#ngram_range=(1,2)
#cv = CountVectorizer(ngram_range=(1,3))

# We only need to count keywords, not all possible terms.
cv = CountVectorizer(ngram_range = (1, 3), vocabulary = keywords)
data = cv.fit_transform(all_patients.TEXT_SPLIT)
tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(data)

# Extract the IDF scores to apply to term frequencies.
word2tfidf = dict(zip(cv.get_feature_names(), tfidf_transformer.idf_))

In [ ]:
#Prints all the tfidf "scores" per keyword as dictionary
print('Positive Keyword IDFs')
positive_dictionary = {}
for word, score in word2tfidf.items():
    if word in positive_keywords: 
        x={word: score}
        print(x)
        positive_dictionary.update(x)

print('')
print('Negative Keyword IDFs')
negative_dictionary = {}
for word, score in word2tfidf.items():
    if word in negative_keywords: 
        x={word: -1*score}
        print(x)
        negative_dictionary.update(x)

In [ ]:
#all_patients['FOUND_STRING_KEYWORDS'].map(lambda x: x.replace('\'','').replace('[','').replace(']',''))
"""
all_patients['FOUND_STRING_KEYWORDS']=all_patients.FOUND_KEYWORD_REPEATS.map(lambda x: ''.join(re.findall(r'\'(\w.+)\'', x))) 
all_patients['FOUND_STRING_KEYWORDS_clean']=all_patients['FOUND_STRING_KEYWORDS'].map(lambda x: x.replace('\'','').replace('[','').replace(']',''))
all_patients['FOUND_STRING_KEYWORDS_clean'].head()
dummies=all_patients['FOUND_STRING_KEYWORDS_clean'].str.get_dummies(sep=', ')
"""

In [ ]:
# dummies.head()

In [ ]:
all_patients.columns

In [ ]:
backup = all_patients

In [ ]:
%%time
# Takes 12 seconds.

#Applies tfidf score to each keyword 
# CK: unclear what this chunk is trying to do vs. what would be ideal.
#all_patients['FOUND_STRING_KEYWORDS']=all_patients.FOUND_KEYWORDS.map(lambda x: ''.join(re.findall(r'\'(\w.+)\'', x))) 
"""
all_patients['FOUND_STRING_KEYWORDS']=all_patients.FOUND_KEYWORD_REPEATS.map(lambda x: ''.join(re.findall(r'\'(\w.+)\'', x))) 
all_patients['FOUND_STRING_KEYWORDS_clean']=all_patients['FOUND_STRING_KEYWORDS'].map(lambda x: x.replace('\'','').replace('[','').replace(']',''))

dummies=all_patients['FOUND_STRING_KEYWORDS_clean'].str.get_dummies(sep=', ')
"""
# positive keywords - only those found in the dataframe
cols_to_update = list(set(positive_dictionary.keys()) & set(all_patients.columns))
print(all_patients[cols_to_update].head())
all_patients[cols_to_update] = all_patients[cols_to_update].astype('float64').mul(pd.Series(positive_dictionary), axis=1)[cols_to_update]
print(all_patients[cols_to_update].head())

# negative keywords - only those found in the dataframe.
cols_to_update = list(set(negative_dictionary.keys()) & set(all_patients.columns))
all_patients[cols_to_update] = all_patients[cols_to_update].astype('float64').mul(pd.Series(negative_dictionary), axis=1)[cols_to_update]

In [ ]:
all_kws = list(set(all_patients.columns) & set(positive_keywords + negative_keywords))
print(all_kws)

In [ ]:
all_patients['total_tfidf_score'] = all_patients[all_kws].sum(axis=1)

In [ ]:
#combine demographics/outcomes and tfidf-scores 
#tfidf_df=pd.concat([all_patients, dummies], axis=1)
tfidf_df = all_patients
tfidf_df.head(5)

In [ ]:
tfidf_df.shape

In [ ]:
tfidf_df.groupby(['SUBJECT_ID', 'HADM_ID', 'day_of_note_from_admit'])['total_tfidf_score'].sum()

In [ ]:
#Creates concat_df, which groups total positive/negative words and tfidf score by day of admission 
num_pos_words = tfidf_df.groupby(['SUBJECT_ID', 'HADM_ID', 'day_of_note_from_admit'])['NUM_POS_KEYWORDS'].sum()
num_neg_words = tfidf_df.groupby(['SUBJECT_ID', 'HADM_ID', 'day_of_note_from_admit'])['NUM_NEG_KEYWORDS'].sum()
tfidf_score = tfidf_df.groupby(['SUBJECT_ID', 'HADM_ID', 'day_of_note_from_admit'])['total_tfidf_score'].mean()
concat_df = pd.concat([num_pos_words, num_neg_words, tfidf_score], axis=1).reset_index()
concat_df.columns=['SUBJECT_ID', 'HADM_ID', 'day_of_note_from_admit', 'total_pos_words',
                  'total_neg_words','tfidf_score']

In [ ]:
concat_df.head(5)

In [ ]:
all_patients.columns.tolist()

In [ ]:
all_patients['admit_date']=pd.to_datetime(all_patients['ADMITTIME']).dt.date

In [ ]:
all_patients.shape

In [ ]:
trunc_all_patients = all_patients[['SUBJECT_ID', 'HADM_ID', #'CATEGORY',
                                 'admit_date', #'CGID', 
                                 'AGE', 'GENDER', 'EXPIRE_FLAG', 'DISCHARGE_LOCATION', 'INSURANCE', 
                                 'LANGUAGE', 'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS', 
                                 'num_mv_days', 'mv_start_date', 'mv_end_date', 'num_trach_days', 'trach_start_date', 
                                 'trach_end_date', 'num_dialysis_days', 'dialysis_start_date', 'dialysis_end_date', 'num_pressor_days', 
                                 'pressors_start_date', 'pressors_end_date', 'DC_TO_DEATH_HOSPICE', 'LENGTH_OF_STAY', 'days_until_mechvent', 
                                 'days_until_dialysis', 'days_until_pressors', 'days_until_trach', 'prior_comorbidities', 'OASIS_score', 'OASIS_prob']].drop_duplicates()

In [ ]:
trunc_all_patients.shape

In [ ]:
#trunc_all_patients2 = trunc_all_patients.groupby(['SUBJECT_ID', 'HADM_ID', 'day_of_note_from_admit'])

In [ ]:
# Merging 
final_df = pd.merge(trunc_all_patients, concat_df, on = ['SUBJECT_ID', 'HADM_ID'], how='left')
final_df['days_until_mechvent'] = pd.to_timedelta(final_df['days_until_mechvent'])
final_df['days_until_dialysis'] = pd.to_timedelta(final_df['days_until_dialysis'])
final_df['days_until_pressors'] = pd.to_timedelta(final_df['days_until_pressors'])
final_df = final_df.drop_duplicates()

In [ ]:
final_df.head(5)

In [ ]:
print(final_df.columns)
print(final_df.shape)

In [ ]:
final_df.sort_values(['SUBJECT_ID', 'HADM_ID']).iloc[0:25]#['SUBJECT_ID','HADM_ID','admit_date', 'AGE', 

In [ ]:
final_df.to_csv('data/tfidfscores_per_day_per_patient_no_dcsum_10.12.csv', index=False)

In [ ]:
#final_df.to_csv('tfidfscores_per_day_per_patient_4.20.csv', index=False)

In [ ]:
#Saves tfidf scores in csv 
csv_file = "data/Negative_Keyword_Dictionary.csv"
with open(csv_file, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in negative_dictionary.items():
        writer.writerow([key, value])

In [ ]:
#Saves tfidf scores in csv 
csv_file = "data/Positive_Keyword_Dictionary.csv"
with open(csv_file, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in positive_dictionary.items():
        writer.writerow([key, value])